# Running SQL Queries Programmatically

- DataFrames can easily be manipulated using SQL queries in PySpark. The `sql()` function on a SparkSession enables applications to run SQL queries programmatically and returns the result as another DataFrame. In this exercise, you'll create a temporary table of the `people_df` DataFrame that you created previously, then construct a query to select the names of the people from the temporary table and assign the result to a new DataFrame.

- Remember, you already have `SparkSession` `spark` and `people_df` DataFrame available in your workspace.

## Instructions

- Create a temporary table `people` that's a pointer to the `people_df` DataFrame.
- Construct a query to select the names of the people from the temporary table `people`.
- Assign the result of Spark's query to a new DataFrame - `people_df_names`.
- Print the top 10 names of the people from `people_df_names` DataFrame.

In [1]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [2]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [3]:
!pwd

/home/talentum/test-jupyter/P2/M3/SM3/3_InteractingwithDataFramesusingPySparkSQL


In [5]:
file_path = "file:///home/talentum/test-jupyter/P2/M3/SM3/3_InteractingwithDataFramesusingPySparkSQL/Dataset/people.csv"

# Create an DataFrame from file_path
people_df = spark.read.csv(file_path, header=True, inferSchema=True)

# Create a temporary table "people"
people_df.createOrReplaceTempView("people")

# Construct a query to select the names of the people from the temporary table "people"
query = '''SELECT name FROM people'''

# Assign the result of Spark's query to people_df_names
people_df_names = spark.sql(query)

# Print the top 10 names of the people
people_df_names.take(10)


[Row(name='Penelope Lewis'),
 Row(name='David Anthony'),
 Row(name='Ida Shipp'),
 Row(name='Joanna Moore'),
 Row(name='Lisandra Ortiz'),
 Row(name='David Simmons'),
 Row(name='Edward Hudson'),
 Row(name='Albert Jones'),
 Row(name='Leonard Cavender'),
 Row(name='Everett Vadala')]

Necessar to Run`Hadoop` and `Hieve` before running sql in pyspark by doing:

**jsp** command will show files for Hadoop
- Haddop
    - HDFS
        - NameNode
        - DataNode
        - SecondaryNameNode
    - YARN
        - ResourceManager
        - NodeManager
   
---

```shell
talentum@talentum-virtual-machine:~$ cat ./Start-Hadoop-Hive.sh 
#!/bin/bash

# Author: Hitesh
# This script starts the HiveMetastore service, Hadoop services (Hdfs and YARN)
# Prerequisit is that this file should be in same directory as that of run-hdfs.sh & run-yarn.sh
# It aslo creates home directory for Hadoop at /user/talentum/ as well as Hive warehouse.

[ -d "/tmp/hadoop-talentum/dfs/data/current" ] && rm -r /tmp/hadoop-talentum/dfs/data/current

[ -d "/tmp/hadoop-talentum/dfs/name/current" ] && rm -r /tmp/hadoop-talentum/dfs/name/current

[ -d "/tmp/hadoop-talentum/dfs/namesecondary/current" ] && rm -r /tmp/hadoop-talentum/dfs/namesecondary/current

hdfs namenode -format

./run-hdfs.sh -s start
echo "-----Running HDFS-----"

./run-yarn.sh -s start
echo "-----Running YARN-----"
echo "-----Running HADOOP-----"

hdfs dfs -mkdir -p /user/talentum/

hdfs dfs -mkdir -p /user/hive/warehouse/

echo "-----Running Hive-----"
./run-hivemetastore.sh -s start&>/dev/null

jps
talentum@talentum-virtual-machine:~$ cat Stop-Hadoop-Hive.sh 
#!/bin/bash

# Author: Hitesh
# This script stops, HiveMetastore service, Hadoop services (Hdfs and YARN)
# Prerequisit is that this file should be in same directory as that of run-hdfs.sh & run-yarn.sh & run-hivemetastore.sh

./run-hivemetastore.sh -s stop
echo "-----Hive Stopped-----"

./run-yarn.sh -s stop
echo "-----YARN Stopped-----"

./run-hdfs.sh -s stop
echo "-----HDFS Stopped-----"
echo "-----HADOOP Stopped-----"

jps

talentum@talentum-virtual-machine:~$ source ./St
Start-Hadoop-Hive.sh  Stop-Hadoop-Hive.sh   
talentum@talentum-virtual-machine:~$ source ./Start-Hadoop-Hive.sh  
talentum@talentum-virtual-machine:~$ source ./Stop-Hadoop-Hive.sh ht
```